In [ ]:
# ==============================================================================
#                      Guía Completa: Data Science en ETL
# ==============================================================================
# Este notebook proporciona ejemplos detallados de prácticas de Data Science en
# cada fase del proceso ETL (Extract, Transform, Load).
#
# Cada sección incluye:
# - Descripción de la práctica.
# - Ejemplo práctico con un escenario de negocio.
# - Código Python ejecutable (con datos de ejemplo simulados cuando es necesario).
# - Herramientas recomendadas.
# - Objetivo clave de la práctica.
#
# ¡Asegúrate de tener las bibliotecas necesarias instaladas!
# Puedes instalarlas con:
# pip install pandas numpy requests sqlalchemy
# (Nota: `psycopg2-binary`, `snowflake-sqlalchemy` y `boto3` son para conexiones reales
# y requerirían instalación. El script usa mocks para ejecutarse sin ellos).
# ==============================================================================

# Importar bibliotecas
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, date, timedelta

# Para simular psycopg2 y sqlalchemy sin una DB real
from sqlalchemy import create_engine, text
from unittest.mock import MagicMock
import os # Para eliminar archivos temporales

# --- Configuración para simulaciones (Mocks) ---
# Estos mocks permiten que el código se ejecute sin bases de datos reales o
# servicios cloud configurados. Si deseas usar conexiones reales, asegúrate
# de tener las bibliotecas correspondientes instaladas y de comentar/eliminar
# las secciones de mockeo relevantes.

class MockCursor:
    def execute(self, query, params=None):
        # print(f"Mock DB: Executing query: {query}") # Descomentar para ver logs del mock
        if "SELECT" in query.upper():
            return self._mock_data()
        return None

    def fetchall(self):
        # Datos simulados para una SELECT en la Práctica 1
        return [
            (1, 101, datetime(2023, 1, 10), 50.00, 'John Doe', 'john.doe@example.com', 'Laptop', 'Electronics', 1000.00),
            (2, 102, datetime(2023, 1, 15), 120.50, 'Jane Smith', 'jane.smith@example.com', 'Mouse', 'Electronics', 25.00),
            (3, 101, datetime(2023, 2, 1), 75.00, 'John Doe', 'john.doe@example.com', 'Keyboard', 'Peripherals', 70.00),
            (4, 103, datetime(2023, 2, 5), 30.00, 'Alice Wonderland', 'alice@example.com', 'Webcam', 'Accessories', 50.00)
        ]

    def close(self):
        # print("Mock DB: Cursor closed.") # Descomentar para ver logs del mock
        pass

    def _mock_data(self):
        pass

class MockConnection:
    def cursor(self):
        return MockCursor()
    def commit(self):
        # print("Mock DB: Committing transaction.") # Descomentar para ver logs del mock
        pass
    def close(self):
        # print("Mock DB: Connection closed.") # Descomentar para ver logs del mock
        pass

def mock_connect(*args, **kwargs):
    # print("Mock DB: Connecting to database.") # Descomentar para ver logs del mock
    return MockConnection()

# Sobrescribir `psycopg2.connect` con nuestro mock si psycopg2 no está disponible
try:
    import psycopg2
    # Si psycopg2 está instalado, se intentará una conexión real.
    # Para forzar el mock incluso con psycopg2 instalado, descomenta la línea:
    # psycopg2.connect = mock_connect
    print("`psycopg2` instalado. Se intentará conexión real, si falla se usarán datos simulados.")
except ImportError:
    print("`psycopg2` no instalado. Usando mock para conexiones a base de datos.")
    import sys
    sys.modules['psycopg2'] = MagicMock()
    sys.modules['psycopg2.extras'] = MagicMock()
    sys.modules['psycopg2'].connect = mock_connect

# Mock para SQLAlchemy `create_engine` y `Connection`
class MockSAConnection:
    def execute(self, statement, parameters=None):
        # print(f"Mock SQLAlchemy: Executing statement: {statement}") # Descomentar para ver logs
        if isinstance(statement, text) and "SELECT" in statement.text.upper():
            return MagicMock(fetchall=lambda: [("2023-11-20",)]) # Simula un resultado para SELECT en DELETE
        return MagicMock()

    def commit(self):
        # print("Mock SQLAlchemy: Committing transaction.") # Descomentar para ver logs
        pass

    def close(self):
        # print("Mock SQLAlchemy: Connection closed.") # Descomentar para ver logs
        pass

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

class MockSAEngine:
    def connect(self):
        return MockSAConnection()

    def dispose(self):
        # print("Mock SQLAlchemy: Engine disposed.") # Descomentar para ver logs
        pass

    # `pd.to_sql` espera que el objeto `con` tenga un método `execute` o `dialect`
    # Simplificamos esto para nuestro mock
    def to_sql(self, df, name, con, if_exists, index):
        # print(f"Mock SQLAlchemy: pd.to_sql called. Table: {name}, if_exists: {if_exists}") # Descomentar para ver logs
        pass # Simula la escritura

# Reemplazar la función real create_engine de SQLAlchemy con nuestro mock
_original_create_engine = create_engine
create_engine = lambda *args, **kwargs: MockSAEngine()

# Reemplazar `pd.DataFrame.to_sql` con un mock si estás usando el engine mock
_original_to_sql_method = pd.DataFrame.to_sql
def mocked_to_sql(df_self, name, con, if_exists='fail', index=True, chunksize=None, dtype=None, method=None):
    if isinstance(con, MockSAEngine) or (hasattr(con, 'connect') and isinstance(con.connect(), MockSAConnection)):
        con.to_sql(df_self, name, con, if_exists, index) # Llama al mock de engine
    else:
        _original_to_sql_method(df_self, name, con, if_exists, index, chunksize, dtype, method)
pd.DataFrame.to_sql = mocked_to_sql


# Mock para boto3 (AWS S3) si no tienes credenciales AWS configuradas
try:
    import boto3
    # Si boto3 está instalado, se intentará una conexión real.
    # Para forzar el mock incluso con boto3 instalado, descomenta la línea:
    # boto3.client = MagicMock(return_value=MagicMock(upload_file=lambda *args, **kwargs: print(f"Mock S3: Uploading file {args[0]} to s3://{args[1]}/{args[2]}")))
    print("`boto3` instalado. Se intentará conexión real, si falla se informará.")
except ImportError:
    print("`boto3` no instalado. Usando mock para operaciones S3.")
    import sys
    sys.modules['boto3'] = MagicMock()
    sys.modules['boto3'].client = MagicMock(return_value=MagicMock(upload_file=lambda *args, **kwargs: print(f"Mock S3: Uploading file {args[0]} to s3://{args[1]}/{args[2]}")))

print("\n--- ¡Mocks configurados para ejecución sin dependencias externas si no están instaladas! ---")
print("Si deseas usar conexiones reales, asegúrate de tener las librerías instaladas y ajusta las secciones de mockeo y credenciales.\n")


# Guía Completa: Data Science en ETL

La etapa de ETL (Extract, Transform, Load) es fundamental en cualquier proyecto de Data Science. Garantiza que los datos estén limpios, consistentes y listos para el análisis y modelado. Este notebook te guiará a través de ejemplos detallados de prácticas de Data Science en cada fase del ETL.

## 1. Extracción (Extract)

**Objetivo:** Recolectar datos de diversas fuentes y formatos, asegurando su integridad.

### Práctica 1: Extracción de Datos de una Base de Datos Relacional

**Descripción:** Conectar a una base de datos SQL para extraer tablas específicas o resultados de consultas complejas.

**Ejemplo Práctico:**
Imagina que eres un Data Scientist en una empresa de e-commerce y necesitas analizar el comportamiento de compra de los clientes. Los datos de transacciones están en una base de datos PostgreSQL.

*   **Fuentes de Datos:** Base de datos PostgreSQL (`orders` table, `customers` table, `products` table).
*   **Datos a Extraer:** `order_id`, `customer_id`, `order_date`, `total_amount` de `orders`, junto con detalles del cliente y producto.

In [ ]:
print("--- FASE 1: EXTRACCIÓN (EXTRACT) ---\n")

### Práctica 1: Extracción de Datos de una Base de Datos Relacional ###

print("### Práctica 1: Extracción de Datos de una Base de Datos Relacional ###")

# Configuración de conexión (usando mock si no hay DB real)
# Si tienes una DB real (ej. PostgreSQL), reemplaza con tus credenciales:
db_config = {
    'host': 'localhost',
    'database': 'testdb',
    'user': 'user',
    'password': 'password'
}

# Query SQL de Extracción
sql_query = """
SELECT
    o.order_id,
    o.customer_id,
    o.order_date,
    o.total_amount,
    c.name AS customer_name,
    c.email AS customer_email,
    p.product_name,
    p.category,
    p.price
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.order_date >= '2023-01-01';
"""

df_raw_orders = pd.DataFrame() # Inicializar

try:
    # Intentar conexión real con psycopg2 si está disponible, si no, usa el mock
    import psycopg2 # Importar aquí para manejar el ImportError del mock
    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()
    cur.execute(sql_query)
    # Obtener nombres de columnas de la descripción del cursor
    column_names = [desc[0] for desc in cur.description]
    raw_data = cur.fetchall()
    df_raw_orders = pd.DataFrame(raw_data, columns=column_names)
    cur.close()
    conn.close()
    print("Extracción de base de datos exitosa (real o simulada).")
except Exception as e:
    print(f"Error conectando a la DB o ejecutando query: {e}. Usando datos simulados.")
    # Simular datos si la conexión falla o el mock está activo
    df_raw_orders = pd.DataFrame({
        'order_id': [1, 2, 3, 4],
        'customer_id': [101, 102, 101, 103],
        'order_date': [datetime(2023, 1, 10), datetime(2023, 1, 15), datetime(2023, 2, 1), datetime(2023, 2, 5)],
        'total_amount': [50.00, 120.50, 75.00, 30.00],
        'customer_name': ['John Doe', 'Jane Smith', 'John Doe', 'Alice Wonderland'],
        'customer_email': ['john.doe@example.com', 'jane.smith@example.com', 'john.doe@example.com', 'alice@example.com'],
        'product_name': ['Laptop', 'Mouse', 'Keyboard', 'Webcam'],
        'category': ['Electronics', 'Electronics', 'Peripherals', 'Accessories'],
        'price': [1000.00, 25.00, 70.00, 50.00]
    })

print("\nPrimeras filas de datos extraídos de la base de datos:")
print(df_raw_orders.head())

# Herramientas a Utilizar: Python (psycopg2, sqlalchemy), Apache Airflow, DBeaver.
# Objetivo: Obtener un conjunto de datos plano y combinado con información de pedidos, clientes y productos.

### Práctica 2: Extracción de Datos de una API Web

**Descripción:** Conectar a una API (Application Programming Interface) para obtener datos en formato JSON o XML.

**Ejemplo Práctico:**
Eres un Data Scientist en una empresa de marketing y necesitas obtener datos de tendencias de búsqueda de Google Trends para un conjunto de palabras clave específicas.

*   **Fuentes de Datos:** Google Trends API (o una API similar).
*   **Datos a Extraer:** Volúmenes de búsqueda por palabra clave, por región y por período de tiempo.

In [ ]:
print("\n### Práctica 2: Extracción de Datos de una API Web ###")

keywords = ['data science', 'machine learning', 'artificial intelligence']
start_date_api = '2023-01-01'
end_date_api = '2023-01-07'
api_key = "YOUR_API_KEY" # Reemplazar si la API real requiere autenticación

all_trends_data = []

print(f"\nExtrayendo datos de API para palabras clave: {', '.join(keywords)}")

# Simulación de respuestas de API
mock_api_responses = {
    'data science': {'trend_score': [80, 85, 82, 88, 90, 87, 91], 'dates': [f'2023-01-0{i+1}' for i in range(7)]},
    'machine learning': {'trend_score': [70, 72, 75, 71, 78, 76, 79], 'dates': [f'2023-01-0{i+1}' for i in range(7)]},
    'artificial intelligence': {'trend_score': [95, 93, 98, 96, 99, 94, 97], 'dates': [f'2023-01-0{i+1}' for i in range(7)]}
}

for keyword in keywords:
    url = f"https://api.example.com/trends?keyword={keyword}&start_date={start_date_api}&end_date={end_date_api}&api_key={api_key}"
    try:
        # En un caso real, usarías requests.get(url)
        # response = requests.get(url)
        # if response.status_code == 200:
        #     trend_data = response.json()
        if keyword in mock_api_responses:
            trend_data = mock_api_responses[keyword]
            all_trends_data.append({'keyword': keyword, 'data': trend_data})
            # print(f"  - Datos para '{keyword}' obtenidos exitosamente (simulado).") # Descomentar para ver logs
        else:
            print(f"  - Error simulado: No se encontraron datos para '{keyword}'.")
    except requests.exceptions.RequestException as e:
        print(f"  - Error de conexión para {keyword}: {e}")
    except json.JSONDecodeError:
        print(f"  - Error decodificando JSON para {keyword}.")

# Convertir los datos a un DataFrame para una mejor visualización
df_trends_list = []
for entry in all_trends_data:
    keyword = entry['keyword']
    data = entry['data']
    for i in range(len(data['dates'])):
        df_trends_list.append({
            'keyword': keyword,
            'date': pd.to_datetime(data['dates'][i]),
            'trend_score': data['trend_score'][i]
        })
df_trends = pd.DataFrame(df_trends_list)

print("\nPrimeras filas de datos de tendencias (API simulada):")
print(df_trends.head())

# Herramientas a Utilizar: Python (requests, json), pytrends, Apache Airflow.
# Objetivo: Obtener datos de tendencias de búsqueda para identificar patrones de interés.

## 2. Transformación (Transform)

**Objetivo:** Limpiar, enriquecer y estructurar los datos para que sean adecuados para el análisis.

### Práctica 3: Limpieza y Estandarización de Datos

**Descripción:** Identificar y corregir errores, inconsistencias y valores faltantes, y estandarizar formatos.

**Ejemplo Práctico:**
Después de extraer los datos de clientes de la base de datos (Ejemplo 1), te das cuenta de que los nombres de los clientes tienen errores tipográficos, los correos electrónicos pueden estar en mayúsculas/minúsculas y las fechas de registro tienen formatos variados.

In [ ]:
print("\n--- FASE 2: TRANSFORMACIÓN (TRANSFORM) ---\n")

### Práctica 3: Limpieza y Estandarización de Datos ###

print("### Práctica 3: Limpieza y Estandarización de Datos ###")

# Datos de entrada simulados (con inconsistencias)
data_customers_raw = {
    'customer_id': [1, 2, 3, 4, 5, 6],
    'customer_name': ["John Doe", "  jane smith  ", "Alice B.", "BOB BROWN", "Charlie D.", "Eve"],
    'customer_email': ["john.doe@example.com", "JANE.SMITH@EXAMPLE.COM", "aliceb@example.com", "bob.brown@example.com", None, "eve@example.com "],
    'registration_date': ["2023-01-15 10:30:00", "Jan 16, 2023", "17/01/2023", "2023-01-18", None, "2023-01-19"]
}
df_customers_raw = pd.DataFrame(data_customers_raw)

print("DataFrame de clientes crudo (con inconsistencias):")
print(df_customers_raw)

# Realizar transformaciones
df_cleaned_customers = df_customers_raw.copy()

# 1. Eliminar espacios en blanco extra de `customer_name` y `customer_email`
df_cleaned_customers['customer_name'] = df_cleaned_customers['customer_name'].str.strip()
df_cleaned_customers['customer_email'] = df_cleaned_customers['customer_email'].str.strip()

# 2. Convertir correos electrónicos a minúsculas
df_cleaned_customers['customer_email'] = df_cleaned_customers['customer_email'].str.lower()

# 3. Unificar formato de fechas y manejar nulos
df_cleaned_customers['registration_date'] = pd.to_datetime(df_cleaned_customers['registration_date'], errors='coerce')
df_cleaned_customers['registration_date'].fillna(datetime.now(), inplace=True) # Imputar con fecha actual
df_cleaned_customers['registration_date'] = df_cleaned_customers['registration_date'].dt.strftime('%Y-%m-%d')

# 4. Imputar valores faltantes en `customer_email` (ej. si después de strip queda None)
df_cleaned_customers['customer_email'].fillna('unknown@example.com', inplace=True)

# 5. (Opcional) Unificar capitalización de nombres (ej. Título)
df_cleaned_customers['customer_name'] = df_cleaned_customers['customer_name'].str.title()

print("\nDataFrame de clientes limpio y estandarizado:")
print(df_cleaned_customers)

# Herramientas a Utilizar: Python (pandas), Talend, OpenRefine.
# Objetivo: Asegurar que los datos de clientes sean consistentes y de alta calidad.

### Práctica 4: Enriquecimiento de Datos y Creación de Features

**Descripción:** Combinar datos de diferentes fuentes y generar nuevas características (features) que puedan ser útiles para el modelado.

**Ejemplo Práctico:**
Tienes los datos de transacciones (Ejemplo 1) y quieres calcular métricas de clientes como el "valor de vida del cliente" (CLV) o la "frecuencia de compra".

In [ ]:
print("\n### Práctica 4: Enriquecimiento de Datos y Creación de Features ###")

# Datos de ejemplo de pedidos (usando df_raw_orders del Ejemplo 1 para más realismo)
df_orders_for_features = df_raw_orders[['customer_id', 'order_date', 'total_amount']].copy()
df_orders_for_features['order_date'] = pd.to_datetime(df_orders_for_features['order_date'])

# Añadir algunos datos adicionales para mostrar más features
new_orders_data = pd.DataFrame({
    'customer_id': [101, 102, 103],
    'order_date': [datetime(2023, 3, 1), datetime(2023, 3, 10), datetime(2023, 3, 15)],
    'total_amount': [90.00, 150.00, 60.00]
})
df_orders_for_features = pd.concat([df_orders_for_features, new_orders_data], ignore_index=True)
df_orders_for_features['order_date'] = pd.to_datetime(df_orders_for_features['order_date']) # Asegurar el tipo datetime

print("DataFrame de órdenes para feature engineering:")
print(df_orders_for_features)

# Fecha actual para calcular antigüedad y recency. Fijamos una fecha para reproducibilidad.
current_analysis_date = pd.to_datetime('2023-04-01')

# 1. Número total de pedidos y monto total gastado por cliente
df_customer_features = df_orders_for_features.groupby('customer_id')['total_amount'].agg(
    num_orders=('total_amount', 'count'),
    total_spent=('total_amount', 'sum')
).reset_index()

# 2. Promedio de gasto por pedido
df_customer_features['avg_spent_per_order'] = df_customer_features['total_spent'] / df_customer_features['num_orders']

# 3. Antigüedad del cliente (días desde el primer pedido)
first_order_date = df_orders_for_features.groupby('customer_id')['order_date'].min().reset_index()
first_order_date.rename(columns={'order_date': 'first_order_date'}, inplace=True)
df_customer_features = pd.merge(df_customer_features, first_order_date, on='customer_id')
df_customer_features['customer_tenure_days'] = (current_analysis_date - df_customer_features['first_order_date']).dt.days

# 4. Calcular Recency (días desde el último pedido)
last_order_date = df_orders_for_features.groupby('customer_id')['order_date'].max().reset_index()
last_order_date.rename(columns={'order_date': 'last_order_date'}, inplace=True)
df_customer_features = pd.merge(df_customer_features, last_order_date, on='customer_id')
df_customer_features['recency_days'] = (current_analysis_date - df_customer_features['last_order_date']).dt.days

# 5. Segmentación RFM básica (Recency, Frequency, Monetary)
# Para el ejemplo, usaremos cuartiles.
# Asegurarse de que haya suficientes valores únicos para qcut.
# Si hay pocos valores, qcut puede fallar. `duplicates='drop'` ayuda a manejar esto.
for col in ['recency_days', 'num_orders', 'total_spent']:
    if df_customer_features[col].nunique() < 4:
        print(f"Advertencia: No hay suficientes valores únicos en '{col}' para un qcut de 4 cuartiles. Asignando valores de manera simplificada.")
        df_customer_features[f'{col}_score'] = pd.qcut(df_customer_features[col], q=df_customer_features[col].nunique(), labels=False, duplicates='drop') + 1
    else:
        labels_q = [1, 2, 3, 4]
        if col == 'recency_days': # Menor recency es mejor, así que invertimos las etiquetas
            labels_q = [4, 3, 2, 1]
        df_customer_features[f'{col}_score'] = pd.qcut(df_customer_features[col], 4, labels=labels_q, duplicates='drop')

df_customer_features['rfm_score'] = df_customer_features['recency_days_score'].astype(str) + \
                                   df_customer_features['num_orders_score'].astype(str) + \
                                   df_customer_features['total_spent_score'].astype(str)


print("\nDataFrame con features de cliente enriquecidas:")
print(df_customer_features)

# Herramientas a Utilizar: Python (pandas, numpy, scikit-learn), Snowflake, Apache Spark.
# Objetivo: Crear features enriquecidas para modelos de segmentación, CLV, churn, etc.

### Práctica 5: Agregación y Resumen de Datos

**Descripción:** Reducir la granularidad de los datos, resumiéndolos a un nivel superior para facilitar el análisis o alimentar dashboards.

**Ejemplo Práctico:**
Necesitas construir un dashboard que muestre las ventas diarias y mensuales por categoría de producto. Los datos de transacciones están a nivel de ítem de pedido.

In [ ]:
print("\n### Práctica 5: Agregación y Resumen de Datos ###")

# Datos de entrada simulados (ítems de pedido y productos)
data_items = {
    'order_date': [datetime(2023, 1, 1), datetime(2023, 1, 1), datetime(2023, 1, 2), datetime(2023, 1, 2), datetime(2023, 2, 1), datetime(2023, 2, 1)],
    'product_id': [10, 20, 10, 30, 20, 10],
    'quantity': [2, 1, 1, 3, 2, 1],
    'price': [10.00, 25.00, 10.00, 5.00, 25.00, 10.00]
}
df_order_items = pd.DataFrame(data_items)

data_products_agg = {
    'product_id': [10, 20, 30],
    'product_name': ['Laptop', 'Mouse', 'Keyboard'],
    'category': ['Electronics', 'Electronics', 'Peripherals']
}
df_products_agg = pd.DataFrame(data_products_agg)

print("DataFrame de ítems de pedido:")
print(df_order_items)
print("\nDataFrame de productos:")
print(df_products_agg)

# 1. Unir `df_order_items` con `df_products_agg` para obtener la categoría
df_merged_sales_agg = pd.merge(df_order_items, df_products_agg, on='product_id')

# 2. Calcular el ingreso por ítem
df_merged_sales_agg['revenue'] = df_merged_sales_agg['quantity'] * df_merged_sales_agg['price']

print("\nDataFrame de ventas con categorías y revenue:")
print(df_merged_sales_agg)

# 3. Agregación diaria por categoría
df_daily_revenue = df_merged_sales_agg.groupby([df_merged_sales_agg['order_date'].dt.date, 'category'])['revenue'].sum().reset_index()
df_daily_revenue.rename(columns={'order_date': 'date'}, inplace=True)
print("\nIngresos Diarios por Categoría:")
print(df_daily_revenue)

# 4. Agregación mensual por categoría
df_merged_sales_agg['month'] = df_merged_sales_agg['order_date'].dt.to_period('M')
df_monthly_revenue = df_merged_sales_agg.groupby(['month', 'category'])['revenue'].sum().reset_index()
print("\nIngresos Mensuales por Categoría:")
print(df_monthly_revenue)

# Herramientas a Utilizar: Python (pandas), Snowflake, Tableau.
# Objetivo: Proporcionar datos resumidos para dashboards de BI.

## 3. Carga (Load)

**Objetivo:** Almacenar los datos transformados en un destino adecuado para su uso final, ya sea un almacén de datos, un lago de datos o un sistema de archivo.

### Práctica 6: Carga Incremental a un Data Warehouse

**Descripción:** Insertar solo los nuevos o modificados registros en el destino, en lugar de recargar todos los datos cada vez. Esto es eficiente para grandes volúmenes de datos que cambian constantemente.

**Ejemplo Práctico:**
Después de transformar los datos de transacciones diarias (Ejemplo 5), necesitas cargarlos en una tabla de hechos (`fact_sales`) en tu Data Warehouse (ej. Snowflake). Solo quieres añadir los registros del día actual, no duplicar los datos históricos.

In [ ]:
print("\n--- FASE 3: CARGA (LOAD) ---\n")

### Práctica 6: Carga Incremental a un Data Warehouse ###

print("### Práctica 6: Carga Incremental a un Data Warehouse ###")

# Datos de ventas diarias transformados (usando df_daily_revenue del Ejemplo 5)
df_daily_sales_to_load = df_daily_revenue.copy()
df_daily_sales_to_load['sale_date'] = df_daily_sales_to_load['date'] # Renombrar para ser consistente
del df_daily_sales_to_load['date'] # Eliminar columna original

# Añadir una columna de métrica adicional para más realismo
df_daily_sales_to_load['num_transactions'] = (df_daily_sales_to_load['total_revenue'] / 10).astype(int) # Simulado
df_daily_sales_to_load.rename(columns={'total_revenue': 'total_revenue'}, inplace=True) # Mantener nombre

print("\nDatos diarios a cargar en el Data Warehouse:")
print(df_daily_sales_to_load)

# Configuración de conexión a Snowflake (o cualquier otra DB SQL)
# Reemplazar con credenciales reales si se desea una conexión real.
user = "YOUR_SNOWFLAKE_USER"
password = "YOUR_SNOWFLAKE_PASSWORD"
account = "YOUR_SNOWFLAKE_ACCOUNT"
warehouse = "YOUR_SNOWFLAKE_WAREHOUSE"
database = "YOUR_SNOWFLAKE_DATABASE"
schema = "YOUR_SNOWFLAKE_SCHEMA"

# Crear la URL de conexión para SQLAlchemy. Usará el mock si no está configurado.
snowflake_url = f"snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}"
engine = create_engine(snowflake_url) # Esto usará nuestro mock si no hay una conexión real

try:
    with engine.connect() as connection:
        # Fecha de los datos que se van a cargar (tomamos la primera fecha del DataFrame)
        load_date = df_daily_sales_to_load['sale_date'].iloc[0].strftime('%Y-%m-%d')
        print(f"\nProcesando carga incremental para la fecha: {load_date}")

        # 1. Eliminar datos existentes para la fecha actual (para idempotencia)
        # Esto es un patrón "delete-then-insert" o "upsert" si la DB lo soporta.
        delete_sql = text(f"DELETE FROM fact_sales WHERE sale_date = '{load_date}';")
        connection.execute(delete_sql)
        connection.commit() # Confirmar la eliminación (en el mock esto no hace nada real)

        print(f"Eliminados datos existentes (si los había) para {load_date} en 'fact_sales' (simulado).")

        # 2. Insertar los nuevos registros
        df_daily_sales_to_load.to_sql('fact_sales', con=connection, if_exists='append', index=False)
        connection.commit() # Confirmar la inserción (en el mock esto no hace nada real)

        print(f"Cargados nuevos datos para {load_date} en 'fact_sales' (simulado).")

except Exception as e:
    print(f"Error durante la carga a Data Warehouse: {e}")
    print("Asegúrate de las credenciales y configuración del DW si intentas una conexión real.")
finally:
    if 'engine' in locals() and engine:
        engine.dispose()
    print("Conexión al Data Warehouse cerrada (simulada o real).")

# Herramientas a Utilizar: Python (pandas, sqlalchemy), Snowflake, Apache Airflow.
# Objetivo: Mantener el Data Warehouse actualizado de manera eficiente.

### Práctica 7: Carga a un Lago de Datos (Data Lake)

**Descripción:** Almacenar datos brutos o semi-estructurados en su formato original o casi original, generalmente en un sistema de archivos distribuido (como HDFS) o almacenamiento de objetos (como S3).

**Ejemplo Práctico:**
Quieres almacenar los datos brutos de la API de Google Trends (Ejemplo 2) en un bucket S3 para futuros análisis o para ser procesados por otras herramientas de Big Data.

In [ ]:
print("\n### Práctica 7: Carga a un Lago de Datos (Data Lake) ###")

# Datos de ejemplo de Google Trends (usando all_trends_data del Ejemplo 2)
print("\nDatos de tendencias a cargar en el Data Lake (Formato JSON):")
print(json.dumps(all_trends_data, indent=2))

# Nombre del bucket y prefijo de la carpeta (ej. basado en la fecha de carga)
bucket_name = 'your-data-lake-bucket-name' # ¡IMPORTANTE! Reemplaza con un bucket S3 real si lo usas.
current_date_str = datetime.now().strftime('%Y-%m-%d')
file_name = f"google_trends_data_{current_date_str}.json"
local_file_path = f"/tmp/{file_name}" # Ruta temporal para el archivo
s3_key = f"raw_data/google_trends/{current_date_str}/{file_name}" # Path en S3

# Crear el directorio /tmp si no existe (importante en algunos entornos como notebooks)
os.makedirs('/tmp', exist_ok=True)

# Guardar los datos a un archivo local temporalmente antes de subir
try:
    with open(local_file_path, 'w') as f:
        json.dump(all_trends_data, f, indent=4)
    print(f"\nArchivo temporal creado en: {local_file_path}")

    # Inicializar cliente S3 (usará el mock si boto3 no está configurado/instalado)
    s3 = boto3.client('s3',
                      aws_access_key_id="YOUR_AWS_ACCESS_KEY",       # Reemplazar con credenciales reales
                      aws_secret_access_key="YOUR_AWS_SECRET_KEY",   # Reemplazar con credenciales reales
                      region_name="YOUR_AWS_REGION")                 # Reemplazar con la región real

    # Subir el archivo a S3
    s3.upload_file(local_file_path, bucket_name, s3_key)
    print(f"Archivo '{file_name}' cargado exitosamente en s3://{bucket_name}/{s3_key} (simulado o real).")

except Exception as e:
    print(f"Error cargando archivo a S3: {e}")
    print("Asegúrate de que las credenciales de AWS estén configuradas (env variables, ~/.aws/credentials) o reemplaza con tus claves y el bucket exista.")
finally:
    # Limpiar el archivo temporal
    if os.path.exists(local_file_path):
        os.remove(local_file_path)
        print(f"Archivo temporal '{local_file_path}' eliminado.")

# Herramientas a Utilizar: Python (boto3), Amazon S3, Apache Airflow.
# Objetivo: Almacenar datos brutos en un formato flexible y escalable.

## Consideraciones Clave para Data Science en ETL:


*   **Calidad de Datos:** Un ETL robusto es la base para modelos de ML precisos. Datos basura (garbage in) resultan en modelos basura (garbage out).
*   **Observabilidad:** Monitorear los pipelines ETL para detectar fallos, latencias o problemas de calidad de datos es fundamental.
*   **Versionado:** Versionar tanto el código ETL como los esquemas de datos y los datos mismos ayuda a la reproducibilidad.
*   **Idempotencia:** Asegurarse de que una re-ejecución del ETL no cause efectos secundarios no deseados (ej. duplicación de datos).
*   **Escalabilidad:** Diseñar el ETL para manejar volúmenes crecientes de datos.
*   **Automatización:** Utilizar orquestadores para automatizar la ejecución programada de los pipelines.
*   **Documentación:** Documentar cada paso del ETL, las fuentes, transformaciones y destinos.

In [ ]:
print("\n--- FIN DEL SCRIPT ---")